In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
from itertools import product
from glob import glob

import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import sbibm
import sbi.analysis as analysis
from sbibm.utils.io import get_tensor_from_csv

from notebookutils import *

## first we create all benchmark cases we need

In [3]:
columns = ['task', 'num_simulations', 'num_observation', 'algorithm', 'num_atoms', 'gamma', 'extra_theta_factor']

# general for nre
tasks = sbibm.get_available_tasks()
num_simulations = [1_000, 10_000, 100_000]
num_observation = list(range(1,11))
num_atoms = [2, 10, 25, 50, 75, 100]
extra_theta_factors = [0]

# for cnre
gammas = 10.0 ** np.arange(-2,3)
gammas = gammas.tolist()

In [4]:
experiments_cnre = list(product(tasks, num_simulations, num_observation, ['CNRE'], num_atoms, gammas, extra_theta_factors))
df_cnre = pd.DataFrame(data=experiments_cnre, columns=columns)

experiments_nre = list(product(tasks, num_simulations, num_observation, ['NRE-B'], num_atoms, [float("nan")], extra_theta_factors))
df_nre = pd.DataFrame(data=experiments_nre, columns=columns)

df_ref = pd.concat([df_cnre, df_nre])
print(len(df_ref))

10800


## then we load what we have to compare

In [5]:
files = [
    # "metrics-0406.csv", 
    "metrics.csv", 
    "metrics-small.csv", 
]
df = pd.concat([pd.read_csv(file, index_col=0) for file in files], ignore_index=True)
df["training_samples_id"] = df["training_samples_root"].map(get_training_samples_id)
print(len(df))
df.head()

warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
warning!
1701


,task,num_simulations,num_observation,seed,max_num_epochs,algorithm,num_atoms,gamma,extra_theta_factor,training_samples_root,...,MMD,MMD_Z,MEDDIST,NLTP,AVG_LOG_RATIO,RT,num_simulations_simulator,path,folder,training_samples_id
0,gaussian_mixture,100000,1,1746250354,300,CNRE,75,100.00,20,extra/010,...,0.083324,0.081130,1.052960,NaN,12.399643,327.643197,100000.0,/home/ben/sci/cnre/benchmarking/multirun/2022-...,49,10
1,gaussian_mixture,100000,1,1224805732,300,CNRE,50,0.10,10,extra/010,...,0.053787,0.049382,1.035793,NaN,9.747178,294.250911,100000.0,/home/ben/sci/cnre/benchmarking/multirun/2022-...,32,10
2,gaussian_mixture,100000,2,2296748511,300,CNRE,25,0.01,20,extra/010,...,0.017185,0.015956,1.044628,NaN,9.180038,303.838053,100000.0,/home/ben/sci/cnre/benchmarking/multirun/2022-...,71,10
3,gaussian_mixture,100000,2,1299984277,300,CNRE,50,0.10,20,extra/010,...,0.014575,0.012991,0.849172,NaN,9.937567,321.064779,100000.0,/home/ben/sci/cnre/benchmarking/multirun/2022-...,83,10
4,gaussian_mixture,100000,2,4055182025,300,CNRE,2,0.10,20,extra/010,...,0.023028,0.021836,1.073073,NaN,5.707378,288.163249,100000.0,/home/ben/sci/cnre/benchmarking/multirun/2022-...,53,10


## difference

In [6]:
index_ref = df_ref.set_index(columns).index
index_df = df.set_index(columns).index
print(len(index_ref.difference(index_df)))

10376
